In [32]:
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer 
import torch.nn.functional as F 
import torch
import pandas as pd
import numpy as np
from math import sqrt
from torch.utils.data import DataLoader

model_ckpt = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_ckpt)


def sdp_attention(query, key, value):
    dim_k = query.size(-1) # dimension component
    sfact = sqrt(dim_k)     
    scores = torch.bmm(query, key.transpose(1,2)) / sfact
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

'''

Attention Class

# nn.linear : apply linear transformation to incoming data
#             y = x * A^T + b
# Ax = b where x is input, b is output, A is weight

# calculate scaled dot product attention matrix
# Requires embedding dimension 
# Each attention head is made of different q,k,v vectors

'''

class Attention(nn.Module):
    
    # initalisation 
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        
        # Define the three vectors
        # input - embed_dim, output - head_dim
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    # main class operation
    def forward(self, hidden_state):
        
        # calculate scaled dot product given a 
        attn_outputs = sdp_attention(
            self.q(hidden_state), 
            self.k(hidden_state), 
            self.v(hidden_state))
        
        return attn_outputs
    

    
'''

Multihead attention class

'''


class multiHeadAttention(nn.Module):
    
    # Config during initalisation
    def __init__(self, config):
        super().__init__()
        
        # model params, read from config file
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        
        # attention head (define only w/o hidden state)
        # each attention head is initialised with embedd/heads head dimension
        self.heads = nn.ModuleList(
            [Attention(embed_dim, head_dim) for _ in range(num_heads)])
        
        # output uses whole embedding dimension for output
        self.out_linear = nn.Linear(embed_dim, embed_dim)

    # Given a hidden state (embeddings)
    # Apply operation for multihead attention
        
    def forward(self, hidden_state):
        
        # for each head embed_size/heads, calculate attention
        heads = [head(hidden_state) for head in self.heads] 
        x = torch.cat(heads, dim=-1) # merge/concat head data together
    
        # apply linear transformation to multihead attension scalar product
        x = self.out_linear(x)
        return x
    
    


class feedForward(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.linear1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    # define layer operations input x
        
    def forward(self, x):    # note must be forward
        x = self.gelu(self.linear1(x))
        x = self.linear2(x)
        x = self.dropout(x)
        return x
    
    
class encoderLayer(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.hidden_size)
        self.norm2 = nn.LayerNorm(config.hidden_size)
        self.attention = multiHeadAttention(config)    # multihead attention layer 
        self.feed_forward = feedForward(config)        # feed forward layer

    def forward(self, x):
        
        # Apply layer norm. to hidden state, copy input into query, key, value
        # Apply attention with a skip connection
        x = x + self.attention(self.norm1(x))
        
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.norm2(x))
        
        return x
    
    
'''

Token + Position Embedding 


'''

class tpEmbedding(nn.Module):
    
    def __init__(self, config):        
        super().__init__()
        
        # token embedding layer
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.hidden_size)
        
        # positional embedding layer
        # config.max_position_embeddings -> max number of positions in text 512 (tokens)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        
        self.norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        
        # Create position IDs for input sequence
        seq_length = input_ids.size(1) # number of tokens
        position_ids = torch.arange(seq_length, dtype=torch.long)[None,:] # range(0,9)
        
        # tensor([[ 1996, 11286,  1997,  1037,  5340,  3392,  2003,  2200,  5931]])
        # tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8]])
        
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        
        # Add normalisation & dropout layers
        embeddings = self.norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings
    
    
    
# full transformer encoder combining the `Embedding` with the ``Embedding` ` layers

class TransformerEncoder(nn.Module):
    
    def __init__(self, config):       
        super().__init__()
        
        # token & positional embedding layer
        self.embeddings = tpEmbedding(config)
        
        # attention & forward feed layer 
        self.layers = nn.ModuleList([encoderLayer(config)
                                     for _ in range(config.num_hidden_layers)])
        
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
    def forward(self, x):
        
        # embeddings layer output
        x = self.embeddings(x)
        
        # cycle through all heads
        for layer in self.layers:
            x = layer(x)
            
        x = x[:, 0, :] # select hidden state of [CLS] token
        x = self.dropout(x)
        return x
    

class TransformerEmbeddings():
    def __init__(self):
        self.model = TransformerEncoder(config)
        self.tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        
    def getTensor(self,text):
        inputs = self.tokenizer(text, 
                   return_tensors="pt",      # pytorc tensor
                   add_special_tokens=False,
                          padding=True) # don't use pad, sep tokens
        return inputs.input_ids
    
    def train(self,data,epochs=100):
        self.model.train()
        tensorData = self.getTensor(data)
        batch_size = 10
        data_loader = DataLoader(tensorData, batch_size=batch_size, shuffle=True)
        for epoch in range(epochs):
            if epoch % 1 == 0:
                print("epoch %s" % (epoch+1))
                for batch in data_loader:
                    self.model.forward(batch)
    
    def getEmbeddings(self,qs):
        with torch.no_grad():
            self.model.eval()
            #CHANGE THIS
            values = []
            for q in qs:
                input = self.getTensor(q)
                x = self.model(input).cpu().detach().numpy()
                values.append(x[0])
            values = np.array(values)
        return values
        
# secDataPath = "../../inputs/SEC/SEC-CompanyTicker.csv"
# data = list(pd.read_csv(secDataPath,index_col=0).companyName[:400])

quoraDataPath = "../../inputs/quora/train.csv"
quoraData = pd.read_csv(quoraDataPath)
quoraData = quoraData[quoraData['is_duplicate'] == 1]
data = list(quoraData.question1[:400])

train = TransformerEmbeddings()
train.train(data)
xb = train.getEmbeddings(data)


epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76
epoch 77
epoch 78
epoch 79
epoch 80
epoch 81
epoch 82
epoch 83
epoch 84
epoch 85
epoch 86
epoch 87
epoch 88
epoch 89
epoch 90
epoch 91
epoch 92
epoch 93
epoch 94
epoch 95
epoch 96
epoch 97
epoch 98
epoch 99
epoch 100


In [34]:
import faiss

class Faiss:
    def __init__(self):
        pass

    def faiss(self,xb):
        d = xb[0].size
        M = 32
        index = faiss.IndexHNSWFlat(d, M)            
        index.hnsw.efConstruction = 40         # Setting the value for efConstruction.
        index.hnsw.efSearch = 16               # Setting the value for efSearch.
        index.add(xb)
        return index
    
    def query(self,index,xq,k=3):
        D, I = index.search(xq, k)   
        return D, I
    
def similaritySearch(index,xq,k=100):
    D,I = Faiss().query(index,xq,k=k)
    I = I[0]
    guesses = [data[guess] for guess in I]
    return guesses

q = list(quoraData.question2[1:2])
xq = train.getEmbeddings(q)    

index = Faiss().faiss(xb)
similaritySearch(index,xq)

['What should I do when my friends betray me?',
 'What can I do to get a job at Microsoft?',
 'What should someone do to overcome anxiety?',
 'What would be a cool way to commit suicide?',
 'What is a good song to lyric prank your best friend?',
 'What would be the best way to quit smoking?',
 'What are the best things to do on Halloween?',
 'What can I eat every day to be more healthy?',
 'What makes a great cup of coffee great?',
 'What do you think about the Bermuda Triangle?',
 'What is the funniest joke you know?',
 'What can be done to reduce the pollution of India?',
 'What’s the best time to have sex?',
 'What are some interesting things to do when bored?',
 'What is the best evidence for a historical Jesus?',
 'What are the easy ways to earn money online?',
 'What Game of Thrones villain would be the most likely to give you mercy?',
 'What is the best way to prepare to IELTS?',
 'What is a narcissistic personality disorder?',
 'What can I do to improve my question on Quora?',


In [35]:
q

['What should I do to be a great geologist?']

In [ ]:
https://www.kaggle.com/code/mahmoudebrahim12/ml-project1-bert